<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Compute Average Mouse Speed (Math.hypot) and Save to CSV
This notebook reads the CSV, computes the average mouse speed (pixels/second) using Python's `math.hypot`, adds it as a new column `avg_speed`, and overwrites the original CSV.

In [1]:
# Parameters
mode = "fake"
input_path = ["C:\\store\\git\\km-stat-activity\\parquet_dataset\\date=2025-04-21\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\part.6.parquet"]
output_path = "C:\\store\\git\\km-stat-activity\\processed\\fake\\profile_guid=011d3524-6fce-4184-bfa3-9707e8c6f18c\\2025-04-21-processed.csv"


In [2]:
# Parameters
# tags: [parameters]

mode = None
input_path = None
output_path = None



<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:

import math
import pandas as pd
from pathlib import Path
import os

# Papermill’den gelen parametreler:
#   mode: "fake" veya "real"
#   input_path: ya bir liste (parquet yolları) ya da bir string (csv yolu)
#   output_path: yazılacak csv’nin tam yolu


if isinstance(input_path, list):
    # Fake: parquet dosyalarını birleştir
    paths = [Path(p) for p in input_path]
    df = pd.concat([pd.read_parquet(p, engine="pyarrow") for p in paths],
                   ignore_index=True)
else:
    # Real: tek CSV oku
    df = pd.read_csv(str(input_path),
                     converters={"x": pd.eval, "y": pd.eval},
                     parse_dates=['start_date_time','end_date_time'])



# 2) Artık df hazır, geri kalan tüm hücreleri olduğu gibi çalıştırabilirsiniz.

# Zaman kolonlarını datetime’a çevir (gerekliyse)
df['start_date_time'] = pd.to_datetime(df['start_date_time'])
df['end_date_time']   = pd.to_datetime(df['end_date_time'])


# 4) Compute average speed using math.hypot
def compute_avg_speed(x, y, t0, t1):
    # Work with the common prefix length
    n = min(len(x), len(y))
    # If we don't have at least two points, speed is zero
    if n < 2:
        return 0.0

    # Sum up the pairwise distances for the first n points
    total_distance = 0.0
    for i in range(1, n):
        dx = x[i] - x[i-1]
        dy = y[i] - y[i-1]
        total_distance += math.hypot(dx, dy)

    # Compute duration in seconds
    duration = (t1 - t0).total_seconds()
    # Guard against zero‐division
    return total_distance / duration if duration > 0 else 0.0

df['avg_speed'] = df.apply(
    lambda row: compute_avg_speed(
        row['x'], row['y'], 
        row['start_date_time'], row['end_date_time']
    ), 
    axis=1
)



# df_result zaten hesaplandı varsayılıyor
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    df_merged = pd.concat(
        [df_existing.reset_index(drop=True), df_result.reset_index(drop=True)],
        axis=1
    )
else:
    df_merged = df_result

df_merged.to_csv(output_path, index=False)

print(f"✔️ Saved CSV with speed: {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: 'None'